In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from pandas import DataFrame,Series
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
import seaborn as sns # plotting library
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from pycaret.regression import *
from sklearn import preprocessing
import plotly.express as px
from tqdm import tqdm

In [ ]:
df = pd.read_csv("covidweather.csv")
df = df.drop(columns = ['UID', 'Population', 'Age-65', 'Income-PC','Deaths', 'Cases', 'New Cases'])
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
#combine
df['Lat'] = ['Lat(' + str(i) for i in df['Lat']]
df['Long'] = ['Long(' + str(i) + ')' for i in df['Long']]

df['time_series'] = df[['County', 'State']].apply(lambda x: ','.join(x), axis=1)
df.drop(['Lat', 'Long'], axis=1, inplace=True)


# extract features from date
df['month'] = [i.month for i in df['Date']]
df['year'] = [i.year for i in df['Date']]
df['day_of_week'] = [i.dayofweek for i in df['Date']]
df['day_of_year'] = [i.dayofyear for i in df['Date']]


df['New Deaths'] = df['New Deaths'].astype(float) 
df.dtypes

In [ ]:
# #visualize
import plotly.express as px
for i in df['time_series'].unique():
    
    subset = df[df['time_series'] == i]
    subset['Moving_Average'] = subset['New Deaths'].rolling(10).mean()
    fig = px.line(subset, x="Date", y=["New Deaths","Moving_Average"], title = i, template = 'plotly_dark')
    fig.show()

In [ ]:
all_ts = df['time_series'].unique()

all_results = []
final_model = {}

for i in tqdm(all_ts):
    
    df_subset = df[df['time_series'] == i]
    
    # initialize setup from pycaret.regression
    s = setup(df_subset, target = 'New Deaths', train_size = 0.90,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ['Date', 'time_series'],
              numeric_features = ['temp', 'humidity', 'windspeed', 'day_of_year', 'year'],
              categorical_features = ['month', 'day_of_week'],
              silent = True, verbose = False, session_id = 123)
    
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose=False)
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['time_series'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # save transformation pipeline and model as pickle file 
    save_model(f, model_name=r'C:\Users\Mubby\OneDrive - University of Kentucky/DOCS/SPRING 2022/DS 711/Capstone python/' + str(i), verbose=False)

In [ ]:
concat_results = pd.concat(all_results,axis=0)
concat_results.head()